# Análisis de Sentimientos

En este archivo se calcula el sentimiento de cada review para luego asignarle un valor (0: negativo, 1: neutral, 2: positivo). 

### Importar Librerías

In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from warnings import filterwarnings
import os
import sys
import pandas as pd
import nltk
import re

# Ignorar todas las advertencias
filterwarnings("ignore")

In [2]:
# descargar el modelo vader_lexicon de NLTK
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Eris\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

In [4]:
# importar función get_file(name_file) de ../functions/EDA.py para leer archivo de reviews
from functions.EDA import get_file

# importar función export(df, project_root, file) de ../functions/ETL.py para guardar el nuevo archivo
from functions.ETL import export


### Importar DataSet user_reviews para realizar análisis de sentimientos

In [5]:
# importamos dataset y visualizamos
df = get_file('user_reviews')
df.head(3)

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...


### Realizar análisis de sentimientos

Para realizar la clasificación de reviews, se utilizó la biblioteca NLTK, que proporciona varias funciones útiles para el procesamiento de lenguaje natural. Se seleccionó el algoritmo VADER, que es un modelo de análisis de opiniones basado en reglas de aprendizaje automático.

<br>
Se decidió no realizar stemming ni lematización dado el entrenamiento del modelo VADER en un amplio corpus de lenguaje natural (incluyendo comentarios informales), lo que le permite manejar adecuadamente palabras en sus formas originales.

<br>
Razones por las que el Stemming y la Lematización pueden no ser necesarios:<br>

    1. VADER maneja formas de palabras 
    2. El corpus de entrenamiento del modelo VADER contiene palabras en sus formas originales
    3. VADER tiene en cuenta signos de puntuación (incluyendo emoticonos hechos con caracteres como ':)')

In [6]:
# Función para preprocesar la review
def preprocess_review(review):
    """
    Esta función preprocesa las reviews para reemplazar símbolos o patrones
    que no son bien interpretados por el análisis de sentimientos.
    """
    if pd.isnull(review):
        return review

    # Reemplazar puntuaciones como 10/10 o 100/100 con términos positivos
    review = re.sub(r'\b\d+/10\b', 'excellent!', review)  # Reemplaza puntuaciones como 10/10
    review = re.sub(r'\b100/100\b', 'perfect!', review)  # Reemplaza puntuaciones 100/100
    
    # Reemplazar corazones ♥ con "I love it"
    review = re.sub(r'♥+', 'I love it!', review)  # + para capturar múltiples corazones
    
    # Si hay muchos puntos suspensivos, podemos ignorarlos
    review = re.sub(r'\.{3,}', '', review)  # Remover tres o más puntos
    
    return review

La función analyze(review, recommend) realiza análisis de sentimientos de la siguiente manera:
- Si el sentimiento es positivo y la reseña positiva, retorna 2 (positivo)
- Si el sentimiento es positivo y la reseña negativa, retorna 1 (neutral)
- Si el sentimiento es negativo y la reseña positiva, retorna 1 (neutral)
- Si el sentimiento es negativo y la reseña negativa, retorna 0 (negativo)
- Si no existe reseña, retorna 1 (neutral)



In [7]:
# se crea una instancia global de SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [8]:
# Define la función para el análisis de sentimientos con recomendación ajustada por componentes
def analyze(review, recommend):
    """
    Esta función recibe una review y un valor de recomendación, y retorna:
    0 si el sentimiento es negativo.
    1 si el sentimiento es neutral.
    2 si el sentimiento es positivo.
    Se da preferencia a la recomendación si existe.
    """
    # Si la review está vacía, retorna 1 (neutral)
    if pd.isnull(review):
        return 1

    # Obtiene los componentes de sentimiento (pos, neg, neu)
    sentiment_scores = sid.polarity_scores(review)
    compound = sentiment_scores['compound']
    pos = sentiment_scores['pos']
    neg = sentiment_scores['neg']
    neu = sentiment_scores['neu']

    # Define un ajuste basado en los componentes del sentimiento
    if recommend is not None:
        if recommend == 1:  # Si recomienda el producto
            if pos > neg and pos > neu:
                compound += pos * 0.5  # Aumenta el impacto positivo si el sentimiento positivo predomina
            elif neu > pos and neu > neg:
                compound += neu * 0.3  # Aumenta un poco si el sentimiento es muy neutral
        elif recommend == 0:  # Si no recomienda el producto
            if neg > pos and neg > neu:
                compound -= neg * 0.5  # Reduce si el sentimiento negativo predomina
            elif neu > pos and neu > neg:
                compound -= neu * 0.3  # Ajuste menor si es neutral

    # Determinar el valor del sentimiento basado en el score ajustado
    if compound >= 0.05:
        sentiment_value = 2  # Positivo
    elif compound <= -0.05:
        sentiment_value = 0  # Negativo
    else:
        sentiment_value = 1  # Neutral

    # Si el usuario recomendó pero el sentimiento es negativo
    if recommend == 1 and sentiment_value == 0:
        return 1  # Lo ajustamos a neutral ya que recomienda pese al sentimiento negativo
    
    # Si el usuario no recomienda pero el sentimiento es positivo
    if recommend == 0 and sentiment_value == 2:
        return 1  # Lo ajustamos a neutral ya que no recomienda pese al sentimiento positivo

    return sentiment_value


In [9]:
# procesa determinados elementos de las reviews para optimizar la interpretación
df['review'] = df.apply(lambda row: preprocess_review(row['review']), axis=1)

df.head(3)

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...


In [10]:
# crea una columna con los resultados del análisis (0: negativo, 1: neutral, 2: positivo) y se visualizan resultados
df['sentiment_analysis'] = df.apply(lambda row: analyze(row['review'], row['recommend']), axis=1)

df.head(3)

,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...,2
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.,2
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...,2


In [12]:
# reemplazar datos en review por datos de sentiment_analysis
df['review'] = df['sentiment_analysis']

# eliminar columna 'sentiment_analysis' y visualizar
df.drop('sentiment_analysis', axis=1, inplace=True)

df.head(3)

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,1,2
1,76561197970982479,2011-07-15,22200,1,2
2,76561197970982479,2011-04-21,43110,1,2


In [11]:
# revisar tipo de dato y verificar nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   posted              59305 non-null  object
 2   item_id             59305 non-null  int64 
 3   recommend           59305 non-null  int64 
 4   review              59275 non-null  object
 5   sentiment_analysis  59305 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.7+ MB


### Exportar DataFrame

In [13]:
# Se utiliza la función export(df, project_root, file) de ../functions/ETL.py para guardar el nuevo archivo
export(df, project_root, 'user_reviews_sentiment_analysis')

Archivos exportados exitosamente.
